In [2]:
import pandas as pd
import os

In [67]:
def tab_paser(dirpath):
    mention_ID = []
    file_ID = []
    language = []
    doc_type = []
    entity_type = []
    mention_type = []
    if_NIL = []
    with open(dirpath,'r') as f_read:
        count = 0
        content = f_read.read().splitlines()
        for line in content:
            if line:
                token = line.split('\t')
                count += 1
                mention_ID.append(count)
#                 print(token)
                file_ID.append(token[3].split(':')[0])
                tmp = token[3][:6]
                if tmp == 'CMN_DF':
                    language.append('CMN')
                    doc_type.append('DF')
                elif tmp == 'CMN_NW':
                    language.append('CMN')
                    doc_type.append('NW')
                elif tmp == 'SPA_DF':
                    language.append('SPA')
                    doc_type.append('DF')
                elif tmp == 'SPA_NW':
                    language.append('SPA')
                    doc_type.append('NW')                    
                elif tmp == 'ENG_DF':
                    language.append('ENG')
                    doc_type.append('DF')
                elif tmp == 'ENG_NW' or tmp == 'NYT_EN':
                    language.append('ENG')
                    doc_type.append('NW')
                else:
                    language.append(0)
                    doc_type.append(0)                    
                    
                entity_type.append(token[5])
                mention_type.append(token[6])
                if token[4][:3] == 'NIL':
                    if_NIL.append("NIL")
                else:
                    if_NIL.append('LINK')
                    
        df = pd.DataFrame({'mention_ID':mention_ID,'doc_ID':file_ID,
                              'language':language,'doc_type':doc_type,
                              'entity_type':entity_type,'mention_type':mention_type, 'if_NIL': if_NIL})
        return df


In [177]:
def etract_tab_filenames(dirpath):
    set_file = set()
    with open(dirpath, 'r') as f_read:
        content =f_read.read().splitlines()
        for line in content:
            token = line.split('\t')
            set_file.add(token[3].split(':')[0])
    return(set_file)

def etract_document_filenames(src_directory):
    set_file = set()
    for (dirpath, dirnames, filenames) in os.walk(src_directory):
            if filenames:
                if filenames[0][-3:] == 'xml':
                    for filename in filenames:
                        set_file.add(filename[:-4])
    return set_file

def if_any_doc_not_in_tab (tab_path, doc_path):
    set_tab = etract_tab_filenames(tab_path)
    set_doc = etract_document_filenames(doc_path)
#version 2017
    print(set_tab - set_doc)
    print('number of docment in golden tab',len(set_tab))
    print('number of docment in corpus',len(set_doc))
#version 2016
#     set_any = set_doc - set_tab
#     if set_any:
#         print('the following documents donot have mention in golden_tab:')
#         print(set_any)
#     else:
#         print('all the docs have mention in golden_tab')

In [136]:
path = '/home/liang/internship/KBP_GOLDEN_DATA/data/LDC2017E03_TAC_KBP_Entity_Discovery_and_Linking_Comprehensive_Training_and_Evaluation_Data_2014-2016_V1.1/docs/tac_kbp_edl_2014-2016_mention_distribution_table.tsv'
df_dist = pd.read_csv(path, sep='\t')

# general inform for 2014-2017

### There are some difference in the input file format:
input data format:
   * 2017 : raw-text-like contain xml tag.
   * 2016 : like in 2017
   * 2015 : *tokenized every word* in xml format.
   * 2014 : like in 2017
    

### Document distribution:

In [217]:
df_1 = df_2017.groupby(['language','doc_type'])['doc_ID'].nunique().to_frame().rename(index=str, columns={"doc_ID": "2017_eval"})
df_2 = df_2016.groupby(['language','doc_type'])['doc_ID'].nunique().to_frame().rename(index=str, columns={"doc_ID": "2016_eval"})
df_3 = df_2015_eval.groupby(['language','doc_type'])['doc_ID'].nunique().to_frame().rename(index=str, columns={"doc_ID": "2015_eval"})
df_4 = df_2015_training.groupby(['language','doc_type'])['doc_ID'].nunique().to_frame().rename(index=str, columns={"doc_ID": "2015_train"})
df_5 = df_2015_pilot.groupby(['language','doc_type'])['doc_ID'].nunique()
print('2015 also contain 5 pilot docments in each 3 langugae, all in NW wise')
pd.concat([df_1, df_2,df_3,df_4], axis=1, sort=False)

2015 also contain 5 pilot docments in each 3 langugae, all in NW wise


2017_eval  2016_eval  2015_eval  2015_train
language doc_type                                             
CMN      DF               84         82         82          63
         NW               83         85         84          84
ENG      DF               84         84         85          83
         NW               83         84         82          85
SPA      DF               83         85         83          47
         NW               83         83         84          82

* 2017 have **90000** document in corpus, but only **500** of them are annotated in golden tab.
* 2016 have **90003** document in corpus, but only **500** of them are annotated in golden tab.


* there is a quit large difference in the golden_data format between **2014** and 15-17. 
* in 2014 there are two data set:
    * 160 training 
    * 138 eval
* and 3 document genres:
    * DF
    * WB
    * NW


but since the webpage of task definiton of 2014 is expired, i can't not find the mapping doc-genre to doc, so i can give a detail statistic like above.

## 2017 DATA

In [178]:
#find the docs have not mentions
tab_path = '/home/liang/internship/EDL-2018/data/tac_kbp_2017_edl_evaluation_gold_standard_entity_mentions.tab'
doc_path = '/home/liang/internship/KBP_GOLDEN_DATA/data/LDC2017E25_TAC_KBP_2017_Evaluation_Source_Corpus_V1.1/data'
if_any_doc_not_in_tab(tab_path, doc_path)

set()
number of docment in golden tab 500
number of docment in corpus 90000


* there are **90,000** documents in the corpus, but only **500** of them have mentinon in the eval golden tab.

In [162]:
# head of statistic
dirpath = '/home/liang/internship/EDL-2018/data/tac_kbp_2017_edl_evaluation_gold_standard_entity_mentions.tab'
df_2017 = tab_paser(dirpath)

### 2017 averaged mention-per-document:

In [179]:
df_2017.groupby(['language','doc_type','doc_ID'])['mention_ID'].count() \
.reset_index(name="mention / doc").groupby(['language','doc_type']).mean()

mention / doc
language doc_type               
CMN      DF            70.785714
         NW            51.807229
ENG      DF            38.226190
         NW            44.626506
SPA      DF            41.578313
         NW            45.313253

### 2017 entity_type distribution:

In [220]:
df_2017.groupby(['entity_type'])['mention_ID'].nunique().reset_index(name="mention count")

,entity_type,mention count
0,FAC,990
1,GPE,8324
2,LOC,1516
3,ORG,5119
4,PER,8424


In [222]:
print('2017 entity_type distributionper entity_type, per mention_type:')
df_2017.groupby(['entity_type','mention_type'])['mention_ID'].nunique().\
to_frame().rename(index=str, columns={"mention_ID": "mention count"})

2017 entity_type distributionper entity_type, per mention_type:


mention count
entity_type mention_type               
FAC         NAM                     359
            NOM                     631
GPE         NAM                    7440
            NOM                     884
LOC         NAM                     705
            NOM                     811
ORG         NAM                    3380
            NOM                    1739
PER         NAM                    6500
            NOM                    1924

<div class="alert alert-success"> generally NAM is much more then NOM</div>

In [223]:
df_2017.groupby(['language','doc_type','if_NIL'])['mention_ID'].nunique().\
to_frame().rename(index=str, columns={"mention_ID": "mention count"})

mention count
language doc_type if_NIL               
CMN      DF       LINK             4413
                  NIL              1533
         NW       LINK             3260
                  NIL              1040
ENG      DF       LINK             1735
                  NIL              1476
         NW       LINK             2837
                  NIL               867
SPA      DF       LINK             1923
                  NIL              1528
         NW       LINK             3059
                  NIL               702

<div class="alert alert-success"> generally LINKED is much more then NIL</div>

## 2016 DATA

* 1. there just eval data in the zip file with **505** document, **not found training set**.
* 2. i also found there are **two docs have not mention in the <gold_standard_entity_mentions.tab>** 

In [117]:
#find the docs have not mentions
tab_path = '/home/liang/internship/test/tac_kbp_2016_edl_evaluation_gold_standard_entity_mentions.tab'
doc_path = '/home/liang/internship/KBP_GOLDEN_DATA/data/LDC2017E03_TAC_KBP_Entity_Discovery_and_Linking_Comprehensive_Training_and_Evaluation_Data_2014-2016_V1.1/data/2016/eval/source_documents/'
if_any_doc_not_in_tab(tab_path, doc_path)

the following documents donot have mention in golden_tab:
{'SPA_NW_001278_20130421_F0001587R', 'NYT_ENG_20130623.0010'}


In [89]:
# head of statistic
dirpath = '/home/liang/internship/EDL-2018/data/tac_kbp_2016_edl_evaluation_gold_standard_entity_mentions.tab'
df_2016 = tab_paser(dirpath)
df_2016.head(5)

,doc_ID,doc_type,entity_type,if_NIL,language,mention_ID,mention_type
0,CMN_DF_000191_20130709_G00A0EBLC,DF,PER,NIL,CMN,1,NAM
1,SPA_DF_001255_20110211_G00A00FWO,DF,GPE,LINK,SPA,2,NAM
2,SPA_DF_001255_20140524_G00A0HWQ9,DF,FAC,NIL,SPA,3,NOM
3,SPA_NW_001278_20130723_F00014O6J,NW,ORG,LINK,SPA,4,NAM
4,CMN_NW_001278_20130624_F00010OMA,NW,ORG,LINK,CMN,5,NAM


### 2016 averaged mention per document:

In [115]:
df_2016.groupby(['language','doc_type','doc_ID'])['mention_ID'].count() \
.reset_index(name="mention / doc").groupby(['language','doc_type']).mean()

mention / doc
language doc_type               
CMN      DF            57.524390
         NW            48.564706
ENG      DF            47.869048
         NW            62.023810
SPA      DF            41.894118
         NW            41.000000

### 2016 entity_type distribution:

In [122]:
df_2016.groupby(['entity_type'])['mention_ID'].nunique().reset_index(name="mention count")

,entity_type,mention count
0,FAC,723
1,GPE,8913
2,LOC,1611
3,ORG,5038
4,PER,8755


In [224]:
print('2016 entity_type distributionper entity_type, per mention_type:')
df_2016.groupby(['entity_type','mention_type'])['mention_ID'].nunique().\
to_frame().rename(index=str, columns={"mention_ID": "mention count"})

2016 entity_type distributionper entity_type, per mention_type:


mention count
entity_type mention_type               
FAC         NAM                     253
            NOM                     470
GPE         NAM                    8011
            NOM                     902
LOC         NAM                    1182
            NOM                     429
ORG         NAM                    3484
            NOM                    1554
PER         NAM                    6787
            NOM                    1968

In [225]:
df_2016.groupby(['language','doc_type','if_NIL'])['mention_ID'].nunique()\
.to_frame().rename(index=str, columns={"mention_ID": "mention count"})

mention count
language doc_type if_NIL               
CMN      DF       LINK             3562
                  NIL              1155
         NW       LINK             3553
                  NIL               575
ENG      DF       LINK             2827
                  NIL              1194
         NW       LINK             4380
                  NIL               830
SPA      DF       LINK             2158
                  NIL              1403
         NW       LINK             2982
                  NIL               421

## 2015 DATA

2015 data have:
    * 444 training sources documents (30,834 mentions)
    * 500 eval sources documents (32,459 mentions)
    * 15 pilot sources documents (686 mentions)

In [12]:
path ='/home/liang/internship/KBP_GOLDEN_DATA/data/LDC2017E03_TAC_KBP_Entity_Discovery_and_Linking_Comprehensive_Training_and_Evaluation_Data_2014-2016_V1.1/data/2015/training'
tab_path = path + '/' + 'tac_kbp_2015_tedl_training_gold_standard_entity_mentions.tab'
doc_path = path + '/' + 'source_docs'
# if_any_doc_not_in_tab(tab_path, doc_path)

In [194]:
# head of statistic
dirpath_eval = '/home/liang/internship/EDL-2018/data/tac_kbp_2015_tedl_evaluation_gold_standard_entity_mentions.tab'
dirpath_training = '/home/liang/internship/EDL-2018/data/tac_kbp_2015_tedl_training_gold_standard_entity_mentions.tab'
dirpath_pilot = '/home/liang/internship/EDL-2018/data/tac_kbp_2015_tedl_pilot_gold_standard_entity_mentions.tab'
df_2015_eval = tab_paser(dirpath_eval)
df_2015_training = tab_paser(dirpath_training)
df_2015_pilot = tab_paser(dirpath_pilot)

In [159]:
df_dist[((df_dist['Year'] == 2015) & (df_dist['Set'] == 'Training'))]

,Year,Set,KB-Link,FAC,GPE,LOC,ORG,PER,TTL,Total
18,2015,Training,CMN DF NIL,6.0,7,10.0,129,1711,NaN,1863
19,2015,Training,CMN DF Non-NIL,59.0,4389,741.0,733,909,NaN,6831
20,2015,Training,CMN NW NIL,9.0,3,2.0,75,290,NaN,379
21,2015,Training,CMN NW Non-NIL,33.0,1933,244.0,601,1232,NaN,4043
22,2015,Training,ENG DF NIL,10.0,10,8.0,33,2646,4.0,2711
23,2015,Training,ENG DF Non-NIL,56.0,1929,194.0,1096,1859,NaN,5134
24,2015,Training,ENG NW NIL,35.0,17,6.0,64,928,7.0,1057
25,2015,Training,ENG NW Non-NIL,72.0,1354,139.0,946,2132,NaN,4643
26,2015,Training,SPA DF NIL,3.0,4,0.0,8,556,NaN,571
27,2015,Training,SPA DF Non-NIL,10.0,382,19.0,133,330,NaN,874


In [160]:
df_dist[((df_dist['Year'] == 2015) & (df_dist['Set'] == 'Eval'))]

,Year,Set,KB-Link,FAC,GPE,LOC,ORG,PER,TTL,Total
30,2015,Eval,CMN DF NIL,3.0,4,6.0,64,1915,NaN,1992
31,2015,Eval,CMN DF Non-NIL,43.0,2679,243.0,728,668,NaN,4361
32,2015,Eval,CMN NW NIL,19.0,6,3.0,78,302,NaN,408
33,2015,Eval,CMN NW Non-NIL,39.0,2155,270.0,1022,819,NaN,4305
34,2015,Eval,ENG DF NIL,26.0,12,17.0,52,2313,102.0,2522
35,2015,Eval,ENG DF Non-NIL,92.0,2457,466.0,1134,1574,NaN,5723
36,2015,Eval,ENG NW NIL,45.0,14,31.0,145,1673,352.0,2260
37,2015,Eval,ENG NW Non-NIL,80.0,2149,240.0,1212,1459,NaN,5140
38,2015,Eval,SPA DF NIL,8.0,1,1.0,23,1169,NaN,1202
39,2015,Eval,SPA DF Non-NIL,8.0,879,86.0,380,366,NaN,1719


In [161]:
df_dist[((df_dist['Year'] == 2015) & (df_dist['Set'] == 'Pilot'))]

,Year,Set,KB-Link,FAC,GPE,LOC,ORG,PER,TTL,Total
12,2015,Pilot,CMN NW NIL,0.0,0,0.0,0,6,NaN,6
13,2015,Pilot,CMN NW Non-NIL,0.0,183,3.0,26,48,NaN,260
14,2015,Pilot,ENG NW NIL,1.0,0,0.0,2,0,13.0,16
15,2015,Pilot,ENG NW Non-NIL,1.0,119,6.0,21,37,NaN,184
16,2015,Pilot,SPA NW NIL,0.0,1,0.0,3,9,NaN,13
17,2015,Pilot,SPA NW Non-NIL,2.0,124,3.0,30,48,NaN,207


## 2014 DATA

* 2014 data only in EN, and consist training and eval two data set.

In [ ]:
dirpath_eval = '/home/liang/internship/EDL-2018/data/tac_kbp_2015_tedl_evaluation_gold_standard_entity_mentions.tab'
dirpath_training = '/home/liang/internship/EDL-2018/data/tac_kbp_2015_tedl_training_gold_standard_entity_mentions.tab'
dirpath_pilot = '/home/liang/internship/EDL-2018/data/tac_kbp_2015_tedl_pilot_gold_standard_entity_mentions.tab'
df_2015_eval = tab_paser(dirpath_eval)
df_2015_training = tab_paser(dirpath_training)
df_2015_pilot = tab_paser(dirpath_pilot)

In [135]:
df_dist[df_dist['Year'] == 2014]

,Year,Set,KB-Link,FAC,GPE,LOC,ORG,PER,TTL,Total
0,2014,Training,ENG DF NIL,NaN,38,NaN,113,966,NaN,1117
1,2014,Training,ENG DF Non-NIL,NaN,273,NaN,172,552,NaN,997
2,2014,Training,ENG WB NIL,NaN,57,NaN,148,426,NaN,631
3,2014,Training,ENG WB Non-NIL,NaN,309,NaN,181,271,NaN,761
4,2014,Training,ENG NW NIL,NaN,111,NaN,394,555,NaN,1060
5,2014,Training,ENG NW Non-NIL,NaN,731,NaN,414,638,NaN,1783
6,2014,Eval,ENG DF NIL,NaN,33,NaN,28,950,NaN,1011
7,2014,Eval,ENG DF Non-NIL,NaN,128,NaN,103,712,NaN,943
8,2014,Eval,ENG WB NIL,NaN,81,NaN,226,639,NaN,946
9,2014,Eval,ENG WB Non-NIL,NaN,421,NaN,311,346,NaN,1078
